In [15]:
test_dict={"dau":"""SELECT 
    metrics_date::date as "time", sum(CAST(metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION)) as "DAU (R1)"
FROM sapphire.sapphire_engagement_metrics_master
WHERE $__timeFilter(metrics_date)
AND application_id IN (${Applications})
      AND period = 1
      AND market = '#Overall#'
      AND os_version = '#Overall#'
      AND device_model = '#Overall#'
      AND client_version = '#Overall#'
      AND client_build_type = '#Overall#'
      AND install_channel_l1 = '#Overall#'
      AND install_channel_l2 = '#Overall#'
      AND install_channel_l3 = '#Overall#'
      AND install_channel_l4 = '#Overall#'
      AND mini_app = ''
      AND first_launch_source = '#Overall#'
      AND launch_source = '#Overall#'
    GROUP BY metrics_date
""",
"""DAU (R7)""":"""select metrics_date::date as "time", AVG("DAU") OVER (ORDER BY metrics_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS "DAU (R7)"
from 
(
SELECT 
    metrics_date, SUM(CAST(metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION)) as "DAU"
FROM sapphire.sapphire_engagement_metrics_master
WHERE metrics_date >= to_char(cast(cast(${__from:date:'YYYY-MM-DD'} as timestamp) + '- 7 DAY' as date), 'yyyy-mm-dd')::TEXT AND metrics_date <= ${__to:date:'YYYY-MM-DD'}
      AND application_id IN (${Applications})
      AND period = 1
      AND market = '#Overall#'
      AND os_version = '#Overall#'
      AND device_model = '#Overall#'
      AND client_version = '#Overall#'
      AND client_build_type = '#Overall#'
      AND install_channel_l1 = '#Overall#'
      AND install_channel_l2 = '#Overall#'
      AND install_channel_l3 = '#Overall#'
      AND install_channel_l4 = '#Overall#'
      AND mini_app = ''
      AND first_launch_source = '#Overall#'
      AND launch_source = '#Overall#'
    GROUP BY metrics_date

) t 

""",
"""retention 7d - all markets""":"""select metrics_date::date as time, sum(cast(device_retention_rate_new_daily->>'return_count' as DOUBLE PRECISION)) / sum(cast(device_retention_rate_new_daily->>'dau' as DOUBLE PRECISION)) as "Retention D7 R1"
from sapphire.sapphire_retention_master
where $__timeFilter(metrics_date)
and offset_day = 7
and application_id in (${Applications})
and market = '#Overall#'
and clientversion = '#Overall#'
and install_source in ('Organic', 'FirstParty', 'RewardsUpsell', 'PaidAds', 'Google Ads ACI', 'Upsell')
and osversion = '#Overall#'
and build = '#Overall#'
and mini_app = ''
and launch_source = '#Overall#'
and device_model = '#Overall#'
group by metrics_date""",
"retention 7d rolling avg - all markets":"""SELECT
    metrics_date::date AS "time", AVG(a."RetentionRate_7D") OVER (ORDER BY metrics_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS "Retention D7 R7"
FROM(
    SELECT 
        metrics_date, sum(cast(device_retention_rate_new_daily->>'return_count' as DOUBLE PRECISION)) / sum(cast(device_retention_rate_new_daily->>'dau' as DOUBLE PRECISION)) as "RetentionRate_7D"
    from sapphire.sapphire_retention_master
    where metrics_date >= cast(cast(${__from:date:'YYYY-MM-DD'} as timestamp) + '- 7 DAY' as TEXT) AND metrics_date <= cast(${__to:date:'YYYY-MM-DD'} as TEXT)
        and offset_day = 7
        and application_id in (${Applications})
        and market = '#Overall#'
        and clientversion = '#Overall#'
        and install_source in ('Organic', 'FirstParty', 'RewardsUpsell', 'PaidAds', 'Google Ads ACI', 'Upsell')
        and osversion = '#Overall#'
        and build = '#Overall#'
        and mini_app = ''
        and launch_source = '#Overall#'
        and device_model = '#Overall#'
    group by metrics_date
) a""",
"""Chat Active DAU (R1)""":"""SELECT 
	metrics_date::date AS "time",
        SUM(CAST(metrics->'ActiveUser'->'dau'->'chat_active_dau' AS DOUBLE PRECISION)) AS "Chat Active DAU (R1)"
FROM codex_metrics.codex_engagement_metrics_master
WHERE metrics_date >= to_char(cast(cast(${__from:date:'YYYY-MM-DD'} as timestamp) + '- 7 DAY' as date), 'yyyy-mm-dd')::TEXT 
  AND metrics_date <= ${__to:date:'YYYY-MM-DD'}
  AND application_id in (${Applications})
	AND id_type= 'reportable_id'
  AND market ='#Overall#'
  AND install_channel_l1 ='#Overall#'
  AND install_channel_l2 ='#Overall#'
	AND first_launch_entry_point ='#Overall#'
	AND period = 1
	AND os_version ='#Overall#'
	AND device_model ='#Overall#'
	AND client_version ='#Overall#'
	AND client_build_type ='#Overall#'
	AND install_channel_l3 ='#Overall#'
	AND install_channel_l4 ='#Overall#'
	AND mini_app ='#Overall#'
	AND first_launch_source ='#Overall#'
GROUP BY metrics_date
ORDER BY metrics_date""",
"""Chat Active DAU (R7)""":"""select  metrics_date::date as "time", AVG("Chat_Active_DAU") OVER (ORDER BY metrics_date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) AS "Chat Active DAU (R7)"
from (
SELECT 
  metrics_date,
  SUM(CAST(metrics->'ActiveUser'->'dau'->'chat_active_dau' AS DOUBLE PRECISION)) AS "Chat_Active_DAU"
FROM codex_metrics.codex_engagement_metrics_master
WHERE metrics_date >= to_char(cast(cast(${__from:date:'YYYY-MM-DD'} as timestamp) + '- 7 DAY' as date), 'yyyy-mm-dd')::TEXT 
  AND metrics_date <= ${__to:date:'YYYY-MM-DD'}
  AND application_id in (${Applications})
	AND id_type= 'reportable_id'
  AND market ='#Overall#'
  AND install_channel_l1 ='#Overall#'
  AND install_channel_l2 ='#Overall#'
	AND first_launch_entry_point ='#Overall#'
	AND period = 1
	AND os_version ='#Overall#'
	AND device_model ='#Overall#'
	AND client_version ='#Overall#'
	AND client_build_type ='#Overall#'
	AND install_channel_l3 ='#Overall#'
	AND install_channel_l4 ='#Overall#'
	AND mini_app ='#Overall#'
	AND first_launch_source ='#Overall#'
GROUP BY metrics_date
) t
"""
}

pgsql_key={"$__timeFilter(metrics_date)":"""
           metrics_date >= '$start_date' -- Adjust the start date as needed
    AND metrics_date <= '$end_date' -- Adjust the end date as needed""",
           "metrics_date::date":"metrics_date",
           """${__from:date:'YYYY-MM-DD'}""":"'$start_date'",
           """${__to:date:'YYYY-MM-DD'}""":"'$end_date'",
           """${Applications}""":"""'2130688B018F4B44BBED68E7A42BBA1E',
        'AE427635ADC245AE973038BCB3D7C21B',
        '4DC5714ABCAD449BA13A9B701A3CF296',
        '4A5B528B59954AAE8725B509A41FBF1A',
        'F185A93DE6764B098D55089F610A3FB8',
        'FC320C411FC12CD4DFBE9A00F3161364'""",
        """${DauMetricsDate}""":"""${dau_metrics_date}"""
           }



In [26]:
import json
with open(r"C:\Users\v-keqichen\Desktop\Python_Pipeline\BingViz-DM\BingVizAdHoc\Analytics\users\v-keqichen\DataJavis\0grafana2pgsql\pgsql_set\daily_chek.json","w") as f:
    f.write(json.dumps(test_dict,indent=4,ensure_ascii=False))

In [16]:
def regular(value:str):
    for origin,target in pgsql_key.items():
        value=value.replace(origin,target)
    return value
for key,value in test_dict.items():
    test_dict[key]=regular(value)

In [18]:
for key,value in test_dict.items():
    print(value)

SELECT 
    metrics_date as "time", sum(CAST(metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION)) as "DAU (R1)"
FROM sapphire.sapphire_engagement_metrics_master
WHERE 
           metrics_date >= '$start_date' -- Adjust the start date as needed
    AND metrics_date <= '$end_date' -- Adjust the end date as needed
AND application_id IN ('2130688B018F4B44BBED68E7A42BBA1E',
        'AE427635ADC245AE973038BCB3D7C21B',
        '4DC5714ABCAD449BA13A9B701A3CF296',
        '4A5B528B59954AAE8725B509A41FBF1A',
        'F185A93DE6764B098D55089F610A3FB8',
        'FC320C411FC12CD4DFBE9A00F3161364')
      AND period = 1
      AND market = '#Overall#'
      AND os_version = '#Overall#'
      AND device_model = '#Overall#'
      AND client_version = '#Overall#'
      AND client_build_type = '#Overall#'
      AND install_channel_l1 = '#Overall#'
      AND install_channel_l2 = '#Overall#'
      AND install_channel_l3 = '#Overall#'
      AND install_channel_l4 = '#Overall#'
      AND mini_app = ''
   

In [19]:
import sys
sys.path.append(r'C:\Users\v-keqichen\Desktop\Python_Pipeline\BingViz-DM\BingVizAdHoc\Analytics\users\v-keqichen\DataJavis')
from src.ToolPGSQL import ToolPGSQL

In [20]:
toolPGSQL=ToolPGSQL(timeout=1000)

init_SQL_Tool, time_out=1000s
PGSQL Connection successful
{'host': 'bingviz-metrics-storage-dup.postgres.database.azure.com', 'port': '5432', 'dbname': 'metrics', 'user': 'bingviz_metrics', 'password': "xb;Rt&F4~]XWj:@r-7.df'QD", 'connect_timeout': 1000}
<connection object at 0x000001E9BA8147B0; dsn: 'user=bingviz_metrics password=xxx connect_timeout=1000 dbname=metrics host=bingviz-metrics-storage-dup.postgres.database.azure.com port=5432', closed: 0>
del
Database connection closed


In [21]:
from string import Template
for key,sql_template in test_dict.items():
    template=Template(sql_template)
    res_pandas=toolPGSQL.execute_v2(template.substitute(start_date='2024-07-01',end_date='2024-07-16',DauMetricsDate='2024-07-16'),debug=False)
    print(res_pandas)

Connection closed, reconnecting...
Database connection closed
PGSQL Connection successful
{'host': 'bingviz-metrics-storage-dup.postgres.database.azure.com', 'port': '5432', 'dbname': 'metrics', 'user': 'bingviz_metrics', 'password': "xb;Rt&F4~]XWj:@r-7.df'QD", 'connect_timeout': 1000}
Reconnected to the database


C:\Users\v-keqichen\Desktop\Python_Pipeline\BingViz-DM\BingVizAdHoc\Analytics\users\v-keqichen\DataJavis\src\ToolPGSQL.py:135: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(sql_query,self.conn)


          time   DAU (R1)
0   2024-07-01  5361140.0
1   2024-07-02  5452217.0
2   2024-07-03  5382707.0
3   2024-07-04  5257640.0
4   2024-07-05  5182366.0
5   2024-07-06  4931339.0
6   2024-07-07  4888932.0
7   2024-07-08  5229366.0
8   2024-07-09  5258479.0
9   2024-07-10  5232876.0
10  2024-07-11  5211443.0
11  2024-07-12  5079268.0
12  2024-07-13  4958985.0
13  2024-07-14  5094135.0
14  2024-07-15  5209354.0
15  2024-07-16  5257983.0
Connection closed, reconnecting...
Database connection closed
PGSQL Connection successful
{'host': 'bingviz-metrics-storage-dup.postgres.database.azure.com', 'port': '5432', 'dbname': 'metrics', 'user': 'bingviz_metrics', 'password': "xb;Rt&F4~]XWj:@r-7.df'QD", 'connect_timeout': 1000}
Reconnected to the database


C:\Users\v-keqichen\Desktop\Python_Pipeline\BingViz-DM\BingVizAdHoc\Analytics\users\v-keqichen\DataJavis\src\ToolPGSQL.py:135: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(sql_query,self.conn)


          time      DAU (R7)
0   2024-06-24  5.744258e+06
1   2024-06-25  5.808299e+06
2   2024-06-26  5.823130e+06
3   2024-06-27  5.776439e+06
4   2024-06-28  5.726701e+06
5   2024-06-29  5.629676e+06
6   2024-06-30  5.562545e+06
7   2024-07-01  5.507814e+06
8   2024-07-02  5.447797e+06
9   2024-07-03  5.380641e+06
10  2024-07-04  5.326538e+06
11  2024-07-05  5.277197e+06
12  2024-07-06  5.246738e+06
13  2024-07-07  5.208049e+06
14  2024-07-08  5.189224e+06
15  2024-07-09  5.161547e+06
16  2024-07-10  5.140143e+06
17  2024-07-11  5.133543e+06
18  2024-07-12  5.118815e+06
19  2024-07-13  5.122764e+06
20  2024-07-14  5.152079e+06
21  2024-07-15  5.149220e+06
22  2024-07-16  5.149149e+06
Connection closed, reconnecting...
Database connection closed
PGSQL Connection successful
{'host': 'bingviz-metrics-storage-dup.postgres.database.azure.com', 'port': '5432', 'dbname': 'metrics', 'user': 'bingviz_metrics', 'password': "xb;Rt&F4~]XWj:@r-7.df'QD", 'connect_timeout': 1000}
Reconnected to th

...

          time  Retention D7 R7
0   2024-06-25         0.079505
1   2024-06-26         0.079653
2   2024-06-27         0.079963
3   2024-06-28         0.080389
4   2024-06-29         0.080505
5   2024-06-30         0.082201
6   2024-07-01         0.087103
7   2024-07-02         0.092935
8   2024-07-03         0.099400
9   2024-07-04         0.105443
10  2024-07-05         0.111221
11  2024-07-06         0.117108
12  2024-07-07         0.121737
13  2024-07-08         0.122487
14  2024-07-09         0.122705
15  2024-07-10         0.123288
Connection closed, reconnecting...
Database connection closed
PGSQL Connection successful
{'host': 'bingviz-metrics-storage-dup.postgres.database.azure.com', 'port': '5432', 'dbname': 'metrics', 'user': 'bingviz_metrics', 'password': "xb;Rt&F4~]XWj:@r-7.df'QD", 'connect_timeout': 1000}
Reconnected to the database


C:\Users\v-keqichen\Desktop\Python_Pipeline\BingViz-DM\BingVizAdHoc\Analytics\users\v-keqichen\DataJavis\src\ToolPGSQL.py:135: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(sql_query,self.conn)


          time  Chat Active DAU (R1)
0   2024-06-24             1365979.0
1   2024-06-25             1382073.0
2   2024-06-26             1361930.0
3   2024-06-27             1314098.0
4   2024-06-28             1221984.0
5   2024-06-29             1103559.0
6   2024-06-30             1118310.0
7   2024-07-01             1207384.0
8   2024-07-02             1233277.0
9   2024-07-03             1219269.0
10  2024-07-04             1197322.0
11  2024-07-05             1127986.0
12  2024-07-06             1046699.0
13  2024-07-07             1065773.0
14  2024-07-08             1163336.0
15  2024-07-09             1182513.0
16  2024-07-10             1144794.0
17  2024-07-11             1081294.0
18  2024-07-12             1007459.0
19  2024-07-13              928124.0
20  2024-07-14              932479.0
21  2024-07-15              991136.0
22  2024-07-16             1022755.0
Connection closed, reconnecting...
Database connection closed
PGSQL Connection successful
{'host': 'bingviz-metr

C:\Users\v-keqichen\Desktop\Python_Pipeline\BingViz-DM\BingVizAdHoc\Analytics\users\v-keqichen\DataJavis\src\ToolPGSQL.py:135: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query(sql_query,self.conn)


          time  Chat Active DAU (R7)
0   2024-06-24          1.365979e+06
1   2024-06-25          1.374026e+06
2   2024-06-26          1.369994e+06
3   2024-06-27          1.356020e+06
4   2024-06-28          1.329213e+06
5   2024-06-29          1.291604e+06
6   2024-06-30          1.266848e+06
7   2024-07-01          1.244191e+06
8   2024-07-02          1.222935e+06
9   2024-07-03          1.202554e+06
10  2024-07-04          1.185872e+06
11  2024-07-05          1.172444e+06
12  2024-07-06          1.164321e+06
13  2024-07-07          1.156816e+06
14  2024-07-08          1.150523e+06
15  2024-07-09          1.143271e+06
16  2024-07-10          1.132632e+06
17  2024-07-11          1.116056e+06
18  2024-07-12          1.098838e+06
19  2024-07-13          1.081899e+06
20  2024-07-14          1.062857e+06
21  2024-07-15          1.038257e+06
22  2024-07-16          1.015434e+06
